<a href="https://colab.research.google.com/github/Msrania/CS392Project-1/blob/main/bonus2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# lib for linear algebra
import numpy as np
import cv2
#from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt
# data visualization
# to see data
import seaborn as sns
from sklearn.linear_model import SGDOneClassSVM
from sklearn import preprocessing
# processing
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import os
import random
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import tensorflow as tf
from sklearn.svm import SVC

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
os.listdir('/content/drive/MyDrive/Colab Notebooks/bonus/images')

['DB1', 'DB2', 'DB3', 'DB4']

In [ ]:
DATADIR = r'/content/drive/MyDrive/Colab Notebooks/bonus/images'

In [ ]:
CATEGORIES = os.listdir(DATADIR)
CATEGORIES

['DB1', 'DB2', 'DB3', 'DB4']

the data before applay any feature extraction methods

In [ ]:
def load_data():
    DATADIR = r'/content/drive/MyDrive/Colab Notebooks/bonus/images'
    data = []
    # loading training data
    for category in CATEGORIES:
        # create path to image of respective expression
        path = os.path.join(DATADIR, category)
        # get the classification  for each expression
        class_num = CATEGORIES.index(category)

        for img in tqdm(os.listdir(path)):
            img_array = cv2.imread(os.path.join(path, img), 0)
            data.append([img_array, class_num])

    return data

In [ ]:
data = load_data()

  0%|          | 0/80 [00:00<?, ?it/s]

In [ ]:
L = 6
W = 4
fig, axes = plt.subplots(L, W, figsize = (10,10))
axes = axes.ravel()

for i in range(0, L * W):
    sample = random.choice(data)
    axes[i].set_title("lable = "+str(CATEGORIES[sample[1]]))
    axes[i].imshow(sample[0], cmap='gray')
    axes[i].axis('off')
plt.subplots_adjust(wspace=0.5)

it's a good idea to apply the Harris corner detector on the entire image dataset before splitting it into training and testing sets. This ensures that the same set of corner features are used for both training and testing, which is important for evaluating the performance of the SVM model on unseen data

In [ ]:
from tqdm import tqdm
def load_data():
    DATADIR = r'/content/drive/MyDrive/Colab Notebooks/bonus/images'
    data = []
    # loading training data
    for category in CATEGORIES:
        # create path to image of respective expression
        path = os.path.join(DATADIR, category)
        # get the classification  for each expression
        class_num = CATEGORIES.index(category)
        for img in tqdm(os.listdir(path)):
            img_path = os.path.join(path, img)
            img_array = cv2.imread(img_path, 0)
            # Resize the image to a consistent size
            img_array = cv2.resize(img_array, (100, 100))
            # Apply the Harris corner detector to the image
            block_size = 2
            ksize = 3
            k = 0.04
            dst = cv2.cornerHarris(img_array, block_size, ksize, k)
            dst_norm = np.empty_like(dst)
            cv2.normalize(dst, dst_norm, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX)
            dst_norm = np.uint8(dst_norm)
            # Add the corner response image and the class label to the data list
            #store the resulting corner response image and the corresponding label as a tuple of the form [dst_norm, class_num]
            feature_vector = dst_norm.flatten()
            data.append([dst_norm, class_num])
    return data

In [ ]:
data = load_data()

In [ ]:
len(data)

In [ ]:
L = 6
W = 4
fig, axes = plt.subplots(L, W, figsize = (10,10))
axes = axes.ravel()

for i in range(0, L * W):
    sample = random.choice(data)
    axes[i].set_title("lable = "+str(CATEGORIES[sample[1]]))
    axes[i].imshow(sample[0], cmap='gray')
    axes[i].axis('off')
plt.subplots_adjust(wspace=0.5)

In [ ]:
# Convert the data list to numpy arrays for use in machine learning
#To use features vector as input to a machine learning algorithm, we need to flatten the 2D array of the corner response image into a 1D feature vector
X = np.array([x[0].flatten() for x in data])
y = np.array([x[1] for x in data])

In [ ]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
# Train an SVM model on the preprocessed data
clf = SVC(kernel='linear', random_state=42).fit(X_train, y_train)


In [ ]:
# Evaluate the model on the test data
accuracy = clf.score(X_test, y_test)
print(f"Test accuracy: {accuracy}")

In [ ]:
from sklearn.metrics import classification_report
# Predict the labels for the test data
y_pred = clf.predict(X_test)

# Compute the classification report
report = classification_report(y_test, y_pred)

# Print the classification report
print(report)

# apply Canny edge

apply canny edge by load_data function, we can modify the function to preprocess each image using the Canny edge detection algorithm

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
img_array = cv2.imread('/content/drive/MyDrive/Colab Notebooks/bonus/images/DB1/101_1.tif', 0)
edges = cv2.Canny(img_array, 100, 200)
fig, axs = plt.subplots(1, 2, figsize=(10, 5))
axs[0].imshow(img_array, cmap='gray')
axs[0].set_title('Original Image')
axs[1].imshow(edges, cmap='gray')
axs[1].set_title('Edges')
plt.show()

In [ ]:
# Define the function to load the data
def load_data():
    DATADIR = r'/content/drive/MyDrive/Colab Notebooks/bonus/images'
    data = []
    # loading training data
    for category in CATEGORIES:
        # create path to image of respective expression
        path = os.path.join(DATADIR, category)
        # get the classification  for each expression
        class_num = CATEGORIES.index(category)

        for img in tqdm(os.listdir(path)):
            img_path = os.path.join(path,img)
            img_array = cv2.imread(img_path, 0)

            # Check if the loaded image is empty
            if img_array is None:
                continue

            # Resize the image to a consistent size
            img_array = cv2.resize(img_array, (100, 100))

            # Apply Canny edge detection to the image
            edges = cv2.Canny(img_array, 100, 100)


# Display the original image and the edge image side by side
            fig, ax = plt.subplots(1, 2, figsize=(10, 5))
            ax[0].imshow(img_array, cmap='gray')
            ax[0].set_title('Original Image')
            ax[1].imshow(edges, cmap='gray')
            ax[1].set_title('Edge Image')
            plt.show()

            # Compute the feature vector for the image
            feature_vector_img = edges.flatten()
            data.append([feature_vector_img, class_num])

    return data

In [ ]:
# Load the data
data = load_data()


In [ ]:
# Extract the feature vectors and labels from the data
X = np.array([x[0].flatten() for x in data])
y = np.array([x[1] for x in data])

In [ ]:
# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Train an SVM classifier
from sklearn.svm import SVC
# Train an SVM classifier
# Train an SVM model on the preprocessed data
clf = SVC(kernel='linear', random_state=42).fit(X_train, y_train)


In [ ]:
import os
print(os.path.exists(DATADIR))

In [ ]:
# Evaluate the model on the test data
accuracy = clf.score(X_test, y_test)
print(f"Test accuracy: {accuracy}")

In [ ]:
from sklearn.metrics import classification_report
# Predict the labels for the test data
y_pred = clf.predict(X_test)

# Compute the classification report
report = classification_report(y_test, y_pred)

# Print the classification report
print(report)

# apply SIFT

In [ ]:
# Define the function to load the data
def load_data():
    DATADIR = r'/content/drive/MyDrive/Colab Notebooks/bonus/images'
    data = []
    # loading training data
    for category in CATEGORIES:
        # create path to image of respective expression
        path = os.path.join(DATADIR, category)
        # get the classification  for each expression
        class_num = CATEGORIES.index(category)

        for img in tqdm(os.listdir(path)):
            img_path = os.path.join(path,img)
            img_array = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

            # Check if the loaded image is empty
            if img_array is None:
                continue

            # Resize the image to a consistent size
            img_array = cv2.resize(img_array, (100, 100))

            # Extract SIFT features from the image
            sift = cv2.xfeatures2d.SIFT_create()
            keypoints, descriptors = sift.detectAndCompute(img_array, None)

            # Visualize the keypoints detected
            img_kp = cv2.drawKeypoints(img_array, keypoints, None)
            plt.imshow(img_kp)
            plt.title('Keypoints for image {}'.format(img_path))
            plt.show()



            # Compute the feature vector for the image
            feature_vector_img = descriptors.flatten()
            data.append([feature_vector_img, class_num])

    return data

In [ ]:
# Load the data
data = load_data()


In [ ]:
# Extract the feature vectors and labels from the data
max_len = max([len(x[0]) for x in data])
X = np.array([np.pad(x[0], (0, max_len - len(x[0])), 'constant') for x in data])
y = np.array([x[1] for x in data])

In [ ]:
# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Train an SVM classifier
from sklearn.svm import SVC
# Train an SVM classifier
# Train an SVM model on the preprocessed data
clf = SVC(kernel='linear', random_state=42).fit(X_train, y_train)

In [ ]:
# Evaluate the model on the test data
accuracy = clf.score(X_test, y_test)
print(f"Test accuracy: {accuracy}")

In [ ]:
from sklearn.metrics import classification_report
# Predict the labels for the test data
y_pred = clf.predict(X_test)

# Compute the classification report
report = classification_report(y_test, y_pred)

# Print the classification report
print(report)

accuracy, precision, and F1 after using the three features as a single feature vector

In [ ]:
# Define the function to load the data
def load_data():
    DATADIR = r'/content/drive/MyDrive/Colab Notebooks/bonus/images'
    data = []
    # loading training data
    for category in CATEGORIES:
        # create path to image of respective expression
        path = os.path.join(DATADIR, category)
        # get the classification  for each expression
        class_num = CATEGORIES.index(category)
        for img in tqdm(os.listdir(path)):
            img_path = os.path.join(path,img)
            img_array = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            # Check if the loaded image is empty
            if img_array is None:
                continue
            # Resize the image to a consistent size
            img_array = cv2.resize(img_array, (100, 100))
            # Extract Harris features from the image
            harris = cv2.cornerHarris(img_array, 2, 3, 0.04)
            harris = cv2.normalize(harris, None, 0, 1, cv2.NORM_MINMAX)
            harris = harris.reshape(-1)
            # Extract Canny edges from the image
            canny = cv2.Canny(img_array, 100, 200)
            canny = canny.reshape(-1)
            # Extract SIFT features from the image
            sift = cv2.xfeatures2d.SIFT_create()
            keypoints, descriptors = sift.detectAndCompute(img_array, None)
            sift = descriptors.flatten()
            # Combine the feature vectors into a single feature vector
            feature_vector = np.concatenate((harris, canny, sift))
            # Visualize the combined feature vector
            plt.plot(feature_vector)
            plt.title('Combined feature vector for image {}'.format(img_path))
            plt.show()
            # Add the feature vector and label to the data list
            data.append([feature_vector, class_num])
    return data

In [ ]:
 data = load_data()


In [ ]:
# Extract the feature vectors and labels from the data
max_len = max([len(x[0]) for x in data])
X = np.array([np.pad(x[0], (0, max_len - len(x[0])), 'constant') for x in data])
y = np.array([x[1] for x in data])

In [ ]:
# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Train an SVM classifier
from sklearn.svm import SVC
# Train an SVM classifier
# Train an SVM model on the preprocessed data
clf = SVC(kernel='linear', random_state=42).fit(X_train, y_train)

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
y_pred = clf.predict(X_test)

acc = accuracy_score(y_test, y_pred)
prec = precision_score(y_test, y_pred, average='weighted')
rec = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

print("Accuracy: {:.2f}%".format(acc*100))
print("Precision: {:.2f}%".format(prec*100))
print("Recall: {:.2f}%".format(rec*100))
print("F1 Score: {:.2f}%".format(f1*100))